In [1]:
from langgraph.graph import StateGraph, START, END
from typing import TypedDict
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langgraph.checkpoint.memory import InMemorySaver

In [2]:
load_dotenv()

True

In [3]:
llm = ChatOpenAI()

In [4]:
class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str

In [5]:
def generate_joke(state: JokeState):
    prompt = f"Generate a joke on the topic {state['topic']}"
    response = llm.invoke(prompt).content
    
    return {'joke': response}

In [6]:
def generate_explanation(state: JokeState):
    prompt = f"Generate an explanation for this joke: {state['joke']}"
    response = llm.invoke(prompt).content
    
    return {'explanation': response}

In [7]:
# Create a Graph
graph = StateGraph(JokeState)

# Add Nodes
graph.add_node("generate_joke", generate_joke)
graph.add_node("generate_explanation", generate_explanation)

# Create Edges
graph.add_edge(START, "generate_joke")
graph.add_edge("generate_joke", "generate_explanation")
graph.add_edge("generate_explanation", END)

# Create In-memory Checkpointer
checkpointer = InMemorySaver()

# Compile the Graph
workflow = graph.compile(checkpointer=checkpointer)

In [8]:
config1 = {"configurable": {'thread_id': '1'}}

initial_state = {
    "topic": "Coffee mug"
}

workflow.invoke(initial_state, config=config1)

{'topic': 'Coffee mug',
 'joke': 'Why did the coffee mug file a police report? Because it was mugged!',
 'explanation': 'This joke is a play on words. In this case, "mugged" is a term that means to be robbed or attacked. So, the coffee mug in the joke filed a police report because it was physically attacked, or "mugged." It adds humor by using the word "mug" in a literal sense to create a silly and unexpected punchline.'}

In [9]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug file a police report? Because it was mugged!', 'explanation': 'This joke is a play on words. In this case, "mugged" is a term that means to be robbed or attacked. So, the coffee mug in the joke filed a police report because it was physically attacked, or "mugged." It adds humor by using the word "mug" in a literal sense to create a silly and unexpected punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-7d7f-6762-8002-1d15018b3cf5'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T11:03:34.953138+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-6cc2-619a-8001-7ad1ba9d2fa4'}}, tasks=(), interrupts=())

In [10]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug file a police report? Because it was mugged!', 'explanation': 'This joke is a play on words. In this case, "mugged" is a term that means to be robbed or attacked. So, the coffee mug in the joke filed a police report because it was physically attacked, or "mugged." It adds humor by using the word "mug" in a literal sense to create a silly and unexpected punchline.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-7d7f-6762-8002-1d15018b3cf5'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T11:03:34.953138+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-6cc2-619a-8001-7ad1ba9d2fa4'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug file a police report? Because it was mugged!'}, next=('generate_explanation',), confi

In [11]:
workflow.invoke({'topic':'Flying Cars'}, config=config1)

{'topic': 'Flying Cars',
 'joke': "Why did the flying car break up with the airplane? \n\nBecause it couldn't handle high altitudes!",
 'explanation': 'This joke plays on the idea that a flying car, which is typically seen as a more low-flying mode of transportation, would not be able to handle the high altitudes that airplanes often travel at. The punchline implies that the flying car ended the relationship with the airplane because it was unable to function properly in high altitude environments.'}

In [12]:
workflow.get_state(config1)

StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car break up with the airplane? \n\nBecause it couldn't handle high altitudes!", 'explanation': 'This joke plays on the idea that a flying car, which is typically seen as a more low-flying mode of transportation, would not be able to handle the high altitudes that airplanes often travel at. The punchline implies that the flying car ended the relationship with the airplane because it was unable to function properly in high altitude environments.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-930b-68d8-8006-dc4a206c11a3'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T11:03:37.212531+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-86a0-64fe-8005-f479a74d144f'}}, tasks=(), interrupts=())

In [13]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car break up with the airplane? \n\nBecause it couldn't handle high altitudes!", 'explanation': 'This joke plays on the idea that a flying car, which is typically seen as a more low-flying mode of transportation, would not be able to handle the high altitudes that airplanes often travel at. The punchline implies that the flying car ended the relationship with the airplane because it was unable to function properly in high altitude environments.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-930b-68d8-8006-dc4a206c11a3'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T11:03:37.212531+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-86a0-64fe-8005-f479a74d144f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car brea

### Time Travel

In [21]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car break up with the airplane? \n\nBecause it couldn't handle high altitudes!", 'explanation': 'This joke plays on the idea that a flying car, which is typically seen as a more low-flying mode of transportation, would not be able to handle the high altitudes that airplanes often travel at. The punchline implies that the flying car ended the relationship with the airplane because it was unable to function properly in high altitude environments.'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-930b-68d8-8006-dc4a206c11a3'}}, metadata={'source': 'loop', 'step': 6, 'parents': {}}, created_at='2025-09-25T11:03:37.212531+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-86a0-64fe-8005-f479a74d144f'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Flying Cars', 'joke': "Why did the flying car brea

In [24]:
workflow.get_state({'configurable': {'thread_id': '1', 'checkpoint_id': '1f099ff4-5eba-6e80-8000-8d757dd87b37'}})

StateSnapshot(values={'topic': 'Coffee mug'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f099ff4-5eba-6e80-8000-8d757dd87b37'}}, metadata={'source': 'loop', 'step': 0, 'parents': {}}, created_at='2025-09-25T11:03:31.726876+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-5eb6-65d8-bfff-ac047ef83765'}}, tasks=(PregelTask(id='8f98044c-b86f-05d1-21d5-ab1aac206004', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result={'joke': 'Why did the coffee mug file a police report? Because it was mugged!'}),), interrupts=())

In [25]:
workflow.invoke(None, {'configurable': {'thread_id': '1', 'checkpoint_id': '1f099ff4-5eba-6e80-8000-8d757dd87b37'}})

{'topic': 'Coffee mug',
 'joke': 'Why did the coffee mug file a police report? Because it was mugged!',
 'explanation': 'This joke is a play on words, using the term "mugged" which can mean to physically attack and rob someone, or to be in possession of a mug or cup. In this case, the joke makes a pun by suggesting that the coffee mug filed a police report because it was physically attacked and robbed, even though in reality, it was just a play on the word "mug."'}

In [26]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug file a police report? Because it was mugged!', 'explanation': 'This joke is a play on words, using the term "mugged" which can mean to physically attack and rob someone, or to be in possession of a mug or cup. In this case, the joke makes a pun by suggesting that the coffee mug filed a police report because it was physically attacked and robbed, even though in reality, it was just a play on the word "mug."'}, next=(), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a005-4322-6d10-8002-c15b745df82c'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T11:11:05.173722+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f09a005-3623-6808-8001-3733abf08dc2'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'topic': 'Coffee mug', 'joke': 'Why did the coffee mug file a police report? Because it was mugg

## Fault Tolerance

In [14]:
import time

In [15]:
# 1. Define the state
class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

In [16]:
# 2. Define steps
def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(10)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

In [17]:
# 3. Build the graph
builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [18]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
❌ Kernel manually interrupted (crash simulated).


In [19]:
print("\n🔁 Re-running the graph to demonstrate fault tolerance...")
final_state = graph.invoke(None, config={"configurable": {"thread_id": 'thread-1'}})
print("\n✅ Final State:", final_state)


🔁 Re-running the graph to demonstrate fault tolerance...
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
✅ Step 3 executed

✅ Final State: {'input': 'start', 'step1': 'done', 'step2': 'done'}


In [20]:
list(graph.get_state_history({"configurable": {"thread_id": 'thread-1'}}))

[StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=(), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff5-8318-652e-8003-44bd3e5aba46'}}, metadata={'source': 'loop', 'step': 3, 'parents': {}}, created_at='2025-09-25T11:04:02.383589+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff5-8316-61b6-8002-5917c5b1f2a3'}}, tasks=(), interrupts=()),
 StateSnapshot(values={'input': 'start', 'step1': 'done', 'step2': 'done'}, next=('step_3',), config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff5-8316-61b6-8002-5917c5b1f2a3'}}, metadata={'source': 'loop', 'step': 2, 'parents': {}}, created_at='2025-09-25T11:04:02.382677+00:00', parent_config={'configurable': {'thread_id': 'thread-1', 'checkpoint_ns': '', 'checkpoint_id': '1f099ff4-cfbf-64c8-8001-d22999354a95'}}, tasks=(PregelTask(id='768cf4a7-5fa3-24b7-b60c-b39f7577d55